In [2]:
%%capture
!pip install langchain-openai
!pip install -q -U peft

In [ ]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U einops
!pip install gdown
!pip install -U flash-attn --no-build-isolation

In [15]:
import re
import json
import torch
import pandas as pd
import numpy as np

from langchain_openai import ChatOpenAI
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
phi_model = AutoModelForCausalLM.from_pretrained(
          "microsoft/phi-2", trust_remote_code=True, device_map="auto"
)

phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
phi_tokenizer.pad_token = phi_tokenizer.eos_token

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
gemma_model = AutoModelForCausalLM.from_pretrained(
          "google/gemma-2b", trust_remote_code=True, device_map="auto"
)

gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", trust_remote_code=True)
gemma_tokenizer.pad_token = gemma_tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    r=128,  # dimension of the updated matrices
    lora_alpha=128,  # parameter for scaling
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.2,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
)
adapters_name = "LucasVitoriano/gemma_2B"

model = PeftModel.from_pretrained(model, adapters_name)
model = model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/628M [00:00<?, ?B/s]

In [4]:
from google.colab import drive, userdata
drive.mount('/content/drive')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

Mounted at /content/drive


In [5]:
# csv_path = '/content/drive/MyDrive/Projetos/Projet DeepL/deepL/data/questions_brazilian_law_ptbr.csv'
# csv_path = '/content/drive/MyDrive/Vida/questions_brazilian_law_ptbr.csv'
csv_path = '/content/drive/MyDrive/questions_brazilian_law_ptbr.csv'

In [ ]:
questions_df = pd.read_csv(csv_path, sep=";")
questions_df

,Perguntas
0,Qual é o principal órgão responsável pela inte...
1,Quais são os três poderes que compõem a estrut...
2,Quais são os direitos fundamentais garantidos ...
3,Quais são os requisitos para alguém ser consid...
4,Qual é o papel do Supremo Tribunal Federal (ST...
...,...
145,Como é estabelecida a organização dos Estados ...
146,Quais são os direitos dos índios previstos na ...
147,O que é o Tribunal de Contas da União e como é...
148,Quais são os princípios que regem a educação n...


In [ ]:
# Tokenize the prompt
def run_questions(model, tokenizer, questions_df):
    respostas = []
    i=0
    for question in questions_df['Perguntas']:
        prompt = f"""

        Você é um especialista em direito brasileiro. Responda à seguinte pergunta em português usando o seu conhecimento sobre a lei brasileira, a constituição e seus artigos,
        o Supremo Tribunal Federal, etc.

        Pergunta: {question}

        Resposta:
        """
        inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to(device)

        # Generate text
        outputs = model.generate(**inputs, max_new_tokens=200)
        text = tokenizer.batch_decode(outputs)[0]

        # Print the generated response
        # print(text)

        respostas.append(text)
        print(i)
        i+=1
    return respostas

In [ ]:
respostas_phi_model = run_questions(model, gemma_tokenizer, questions_df)

0
1
2
3


In [ ]:
df_respostas = pd.DataFrame(respostas_phi_model,columns=['Response_Gemma_FT'])
df_respostas

,Response_Gemma_FT
0,<bos>\n\n Você é um especialista em dir...
1,<bos>\n\n Você é um especialista em dir...
2,<bos>\n\n Você é um especialista em dir...
3,<bos>\n\n Você é um especialista em dir...
4,<bos>\n\n Você é um especialista em dir...
...,...
145,<bos>\n\n Você é um especialista em dir...
146,<bos>\n\n Você é um especialista em dir...
147,<bos>\n\n Você é um especialista em dir...
148,<bos>\n\n Você é um especialista em dir...


In [ ]:
df_respostas.to_pickle('/content/drive/MyDrive/Response_Gemma_FT.pkl')

## Evaluation

In [6]:
df_phi_base = pd.read_pickle('/content/drive/MyDrive/Projetos/Projet DeepL/deepL/data/Responses/Response_Phi_Base.pkl')
df_phi_ft = pd.read_pickle('/content/drive/MyDrive/Projetos/Projet DeepL/deepL/data/Responses/Response_Phi_FT.pkl')
df_gemma_base = pd.read_pickle('/content/drive/MyDrive/Projetos/Projet DeepL/deepL/data/Responses/Response_Gemma_Base.pkl')
df_gemma_ft = pd.read_pickle('/content/drive/MyDrive/Projetos/Projet DeepL/deepL/data/Responses/Response_Gemma_FT.pkl')

In [7]:
df_responses = pd.concat([df_phi_base, df_phi_ft, df_gemma_base, df_gemma_ft], axis=1)
df_responses

,Response_Phi_Base,Response_Phi_FT,Response_Gemma_Base,Response_Gemma_FT
0,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
1,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
2,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
3,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
4,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
...,...,...,...,...
145,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
146,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
147,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...
148,\n\n Você é um especialista em direito ...,\n\n Você é um especialista em direito ...,<bos>\n\n Você é um especialista em dir...,<bos>\n\n Você é um especialista em dir...


In [8]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,model='gpt-3.5-turbo')

### Phi FT x Gemma FT

In [9]:
response_list = []
for response1, response2 in zip (df_responses['Response_Phi_FT'], df_responses['Response_Gemma_FT']):
  prompt = f'''
    Estou avaliando dois modelos de linguagem para decidir qual é o melhor, e preciso da sua ajuda.
    Mostrarei o prompt enviado por mim e a resposta dos dois modelos, e preciso que você me responda dizendo qual modelo teve a resposta mais adequada.

    Modelo 1:
    "{response1}"

    Modelo 2:
    "{response2}"


    Qual modelo teve a melhor resposta? Responda com APENAS um dígito, indicando o número do modelo (1 ou 2). Não há necessidade de justificar.
    (Exemplo: a resposta deve ser "1" se o Modelo 1 for escolhido, e "2" se o contrário)
  '''
  response = llm.invoke(prompt).content
  print(response)
  response_list.append(response)

Eu escolho o Modelo 2, pois ele forneceu uma resposta direta e correta à pergunta, enquanto o Modelo 1 apresentou uma resposta confusa e repetitiva.
Modelo 2, pois apresentou uma resposta mais completa e clara, identificando corretamente os três poderes da estrutura política brasileira e suas respectivas funções.
Modelo 2, pois apesar de conter alguns erros de digitação, apresentou uma resposta mais coerente e relevante sobre os direitos fundamentais garantidos pela Constituição Federal brasileira.
Modelo 2, pois apresentou uma resposta mais completa e relevante, abordando os requisitos para alguém ser considerado culpado em um processo criminal no Brasil, enquanto o Modelo 1 apresentou uma resposta sem sentido.
Modelo 1, pois apesar de conter erros gramaticais, apresentou uma resposta mais coerente e com informações relevantes sobre o papel do Supremo Tribunal Federal no sistema judiciário brasileiro.
Modelo 2, pois apesar de conter repetições e falta de coesão, apresenta uma resposta

In [10]:
def extract_first_number(text):
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())
    else:
        return None

response_list = [extract_first_number(i) for i in response_list]

In [11]:
len([i for i in response_list if i==2])

122

### Gemma Base x Gemma FT

In [ ]:
response_list = []
for response1, response2 in zip (df_responses['Response_Gemma_Base'], df_responses['Response_Gemma_FT']):
  prompt = f'''
    Estou avaliando dois modelos de linguagem para decidir qual é o melhor, e preciso da sua ajuda.
    Mostrarei o prompt enviado por mim e a resposta dos dois modelos, e preciso que você me responda dizendo qual modelo teve a resposta mais adequada.

    Modelo 1:
    "{response1}"

    Modelo 2:
    "{response2}"


    Qual modelo teve a melhor resposta? Responda com APENAS um dígito, indicando o número do modelo (1 ou 2). Não há necessidade de justificar.
    (Exemplo: a resposta deve ser APENAS "1" se o Modelo 1 for escolhido, e APENAS "2" se o Modelo 2 for escolhido)
  '''
  response = llm.invoke(prompt).content
  print(response)
  response_list.append(response)

In [18]:
response_list = [extract_first_number(i) for i in response_list]
len([i for i in response_list if i==2])

86

## Human Feedback

In [19]:
file_path = '/content/drive/MyDrive/Projetos/Projet DeepL/deepL/data/results.xlsx'
df = pd.read_excel(file_path)

choices = []

for index, row in df.iterrows():
    print(f"Text 1: {row['Response_Gemma_Base']}\nText 2: {row['Response_Gemma_FT']}\n")

    choice = input("Which text is better? Type 1 for Text 1, 2 for Text 2: ")

    while choice not in ['1', '2']:
        print("Invalid input. Please type 1 or 2.")
        choice = input("Which text is better? Type 1 for Text 1, 2 for Text 2: ")

    choices.append(int(choice))

    print("\n" + "-"*50 + "\n")

print("Your choices have been recorded:", choices)

Text 1: <bos>

        Você é um especialista em direito brasileiro. Responda à seguinte pergunta em português usando o seu conhecimento sobre a lei brasileira, a constituição e seus artigos,
        o Supremo Tribunal Federal, etc.

        Pergunta: Qual é o principal órgão responsável pela interpretação e aplicação das leis no Brasil?

        Resposta:
        A) Supremo Tribunal Federal
        B) Tribunal de Justiça
        C) Tribunal de Contas
        D) Tribunal de Justiça Federal
        E) Tribunal de Contas do Distrito Federal

        Resposta:
        A) Supremo Tribunal Federal
        B) Tribunal de Justiça
        C) Tribunal de Contas
        D) Tribunal de Justiça Federal
        E) Tribunal de Contas do Distrito Federal

        Resposta:
        A) Supremo Tribunal Federal
        B) Tribunal de Justiça
        C) Tribunal de Contas
        D) Tribunal de Justiça Federal
        E) Tribunal de Contas do Distrito Federal

        Resposta:
        A) Supremo Tribuna

KeyboardInterrupt: Interrupted by user